In [11]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not set.")

# Load PDF documents
documents = SimpleDirectoryReader(input_files=["retail.pdf"]).load_data()

# Create a vector index for querying
index = VectorStoreIndex.from_documents(documents)

# Create a query engine
query_engine = index.as_query_engine()

# Extract requirements from the PDF
response = query_engine.query("Summarize the key requirements for the Retail Inventory Management System.")
customer_requirements = response.response
print("Extracted Requirements:", customer_requirements)

Extracted Requirements: The key requirements for the Retail Inventory Management System include features such as real-time tracking of inventory levels, integration with point-of-sale systems, support for multiple locations, ability to set reorder points, generate reports for sales and inventory, and provide alerts for low stock levels.


In [12]:
# Initialize LLM for LangChain
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=api_key)

# Agent Prompts
CUSTOMER_PROMPT = """You are a Customer Agent for a retail business. Your role is to provide clear requirements for the Retail Inventory Management System based on provided details. Relay these to the Product Owner and review sprint deliverables, providing feedback for improvements. Requirements: {requirements}"""

PRODUCT_OWNER_PROMPT = """You are a Product Owner in a Scrum team. Translate customer requirements into a product backlog with prioritized user stories. Collaborate with the Scrum Master to define sprint goals and incorporate customer feedback after each sprint. Deliverables: product backlog, sprint goals."""

SCRUM_MASTER_PROMPT = """You are a Scrum Master facilitating Agile ceremonies. Create sprint backlogs based on the product backlog, ensure team collaboration, and remove blockers. Deliverables: sprint backlog, sprint summary."""

DEVELOPER_PROMPT = """You are a Developer Agent. Implement features from the sprint backlog, writing modular code for the Retail Inventory Management System. Deliverables: feature implementations, progress report."""

QA_ENGINEER_PROMPT = """You are a QA Engineer Agent. Develop and execute test cases for sprint deliverables, ensuring functionality and quality. Deliverables: test cases, test results, defect report."""

UI_UX_DESIGNER_PROMPT = """You are a UI/UX Designer Agent. Create wireframes and designs for the sprint's user stories, ensuring usability and accessibility. Deliverables: wireframes, design prototypes."""

BUSINESS_ANALYST_PROMPT = """You are a Business Analyst Agent. Refine user stories with detailed requirements and acceptance criteria. Deliverables: refined user stories, requirements document."""

DEVOPS_ENGINEER_PROMPT = """You are a DevOps Engineer Agent. Set up CI/CD pipelines and deploy sprint deliverables. Deliverables: deployment plan, pipeline configuration."""

ARCHITECT_PROMPT = """You are an Architect Agent. Design the system architecture for sprint features, ensuring scalability. Deliverables: architecture diagram, technical specifications."""

SECURITY_ENGINEER_PROMPT = """You are a Security Engineer Agent. Audit sprint deliverables for vulnerabilities and recommend mitigations. Deliverables: security report, mitigation plan."""

TECHNICAL_WRITER_PROMPT = """You are a Technical Writer Agent. Document sprint deliverables, including user guides and release notes. Deliverables: documentation, release summary and provide a final report after three sprints."""

In [14]:
from typing import TypedDict, List, Dict, Optional

class ScrumState(TypedDict):
    customer_requirements: str
    product_backlog: str
    sprint_backlog: str
    sprint_number: int
    sprint_deliverables: Dict[str, str]
    customer_feedback: str
    current_phase: str
    conversation_history: List[Dict[str, str]]
    completion_status: str

In [ ]:
def customer_phase(state: ScrumState) -> ScrumState:
    messages = [
        SystemMessage(content=CUSTOMER_PROMPT.format(requirements=state["customer_requirements"])),
        HumanMessage(content="Provide the requirements for the Retail Inventory Management System.")
    ]
    response = llm.invoke(messages)
    state["customer_requirements"] = response.content
    state["conversation_history"].append({"Customer": response.content})
    state["current_phase"] = "product_owner"
    print("Customer Requirements:", response.content)
    return state

def product_owner_phase(state: ScrumState) -> ScrumState:
    messages = [
        SystemMessage(content=PRODUCT_OWNER_PROMPT),
        HumanMessage(content=f"Create a product backlog based on these requirements:\n{state['customer_requirements']}")
    ]
    response = llm.invoke(messages)
    state["product_backlog"] = response.content
    state["conversation_history"].append({"Product Owner": response.content})
    state["current_phase"] = "scrum_master"
    print("Product Backlog:", response.content)
    return state

def scrum_master_phase(state: ScrumState) -> ScrumState:
    messages = [
        SystemMessage(content=SCRUM_MASTER_PROMPT),
        HumanMessage(content=f"Create a sprint backlog for sprint {state['sprint_number']} based on this product backlog:\n{state['product_backlog']}")
    ]
    response = llm.invoke(messages)
    state["sprint_backlog"] = response.content
    state["conversation_history"].append({"Scrum Master": response.content})
    state["current_phase"] = "development"
    print(f"Sprint {state['sprint_number']} Backlog:", response.content)
    return state

def development_phase(state: ScrumState) -> ScrumState:
    team_agents = [
        ("Developer", DEVELOPER_PROMPT),
        ("QA Engineer", QA_ENGINEER_PROMPT),
        ("UI/UX Designer", UI_UX_DESIGNER_PROMPT),
        ("Business Analyst", BUSINESS_ANALYST_PROMPT),
        ("DevOps Engineer", DEVOPS_ENGINEER_PROMPT),
        ("Architect", ARCHITECT_PROMPT),
        ("Security Engineer", SECURITY_ENGINEER_PROMPT),
        ("Technical Writer", TECHNICAL_WRITER_PROMPT)
    ]
    state["sprint_deliverables"] = {}
    for agent_name, prompt in team_agents:
        messages = [
            SystemMessage(content=prompt),
            HumanMessage(content=f"Perform your role for sprint {state['sprint_number']} based on this sprint backlog:\n{state['sprint_backlog']}")
        ]
        response = llm.invoke(messages)
        state["sprint_deliverables"][agent_name] = response.content
        state["conversation_history"].append({agent_name: response.content})
        print(f"{agent_name} Deliverables:", response.content)
    state["current_phase"] = "customer_review"
    return state

def customer_review_phase(state: ScrumState) -> ScrumState:
    deliverables_summary = "\n".join([f"{agent}: {content}" for agent, content in state["sprint_deliverables"].items()])
    messages = [
        SystemMessage(content=CUSTOMER_PROMPT.format(requirements=state["customer_requirements"])),
        HumanMessage(content=f"Review the deliverables for sprint {state['sprint_number']}:\n{deliverables_summary}\nProvide feedback.")
    ]
    response = llm.invoke(messages)
    state["customer_feedback"] = response.content
    state["conversation_history"].append({"Customer Feedback": response.content})
    state["sprint_number"] += 1
    state["current_phase"] = "product_owner" if state["sprint_number"] <= 3 else "completed"
    print(f"Customer Feedback (Sprint {state['sprint_number']-1}):", response.content)
    return state

def final_report_phase(state: ScrumState) -> ScrumState:
    history_summary = "\n".join([f"{key}: {value}" for entry in state["conversation_history"] for key, value in entry.items()])
    messages = [
        SystemMessage(content=TECHNICAL_WRITER_PROMPT.format(sprint_number="final")),
        HumanMessage(content=f"Generate a final report summarizing the three sprints based on this history:\n{history_summary}")
    ]
    response = llm.invoke(messages)
    print("\n=== Final Report for Three Sprints ===")
    print(response.content)
    state["conversation_history"].append({"Technical Writer Final Report": response.content})
    return state

In [16]:
def run_workflow():
    # Initialize state
    initial_state = ScrumState(
        customer_requirements=customer_requirements,  # From LlamaIndex
        product_backlog="",
        sprint_backlog="",
        sprint_number=1,
        sprint_deliverables={},
        customer_feedback="",
        current_phase="customer",
        conversation_history=[],
        completion_status="in_progress"
    )

    state = initial_state
    while state["completion_status"] != "completed":
        if state["current_phase"] == "customer":
            state = customer_phase(state)
        elif state["current_phase"] == "product_owner":
            state = product_owner_phase(state)
        elif state["current_phase"] == "scrum_master":
            state = scrum_master_phase(state)
        elif state["current_phase"] == "development":
            state = development_phase(state)
        elif state["current_phase"] == "customer_review":
            state = customer_review_phase(state)
    
    print("Workflow completed after 3 sprints!")
    return state

# Run the workflow
final_state = run_workflow()

Customer Requirements: Here are the key requirements for the Retail Inventory Management System:

1. **Real-Time Inventory Tracking**:
   - The system must provide real-time updates on inventory levels across all locations.
   - It should allow users to view current stock levels, including quantities on hand, reserved, and on order.

2. **Integration with Point-of-Sale (POS) Systems**:
   - The system must seamlessly integrate with existing POS systems to ensure accurate inventory updates with each sale.
   - It should support multiple POS systems and allow for easy configuration.

3. **Support for Multiple Locations**:
   - The system must manage inventory across multiple retail locations.
   - Users should be able to view and manage inventory levels for each location individually or collectively.

4. **Reorder Point Management**:
   - The system should allow users to set reorder points for each product.
   - It must automatically generate purchase orders when stock levels fall below 

KeyboardInterrupt: 

In [17]:
from langgraph.graph import StateGraph, END

# Initialize the graph
workflow = StateGraph(ScrumState)

# Add nodes
workflow.add_node("customer", customer_phase)
workflow.add_node("product_owner", product_owner_phase)
workflow.add_node("scrum_master", scrum_master_phase)
workflow.add_node("development", development_phase)
workflow.add_node("customer_review", customer_review_phase)

# Add edges
workflow.add_edge("customer", "product_owner")
workflow.add_edge("product_owner", "scrum_master")
workflow.add_edge("scrum_master", "development")
workflow.add_edge("development", "customer_review")

# Conditional edge for sprint iteration
def route_after_review(state: ScrumState) -> str:
    if state["sprint_number"] <= 3:
        return "product_owner"
    return END

workflow.add_conditional_edges("customer_review", route_after_review, {"product_owner": "product_owner", END: END})

# Set entry point
workflow.set_entry_point("customer")

# Compile the graph
graph = workflow.compile()

# Run the graph
final_state = graph.invoke(initial_state)

NameError: name 'initial_state' is not defined

In [ ]:
# Test query to verify the index
response = query_engine.query("What was the feedback for Sprint 1?")
print("Query Response:", response)